In [1]:
import hydra
import torch
from tqdm.notebook import tqdm
from lib.utils.config import load_config
from lib.optimizer.framework import NeuralOptimizer
from lib.data.loader import load_intrinsics
from lib.data.loader import load_intrinsics
from lib.rasterizer import Rasterizer
from lib.renderer.renderer import Renderer
from lib.renderer.camera import Camera
from lib.tracker.timer import TimeTracker
from collections import defaultdict
import pandas as pd


def path_to_abblation(path):
    return "_".join(path.split("/")[-3].split("_")[1:])


def eval_iterations(optimizer, datamodule, N: int = 3):
    optimizer.max_iters = N
    optimizer.max_optims = 1
    time_tracker = TimeTracker()
    n_threshold =  optimizer.c_module.n_threshold
    d_threshold = optimizer.c_module.d_threshold
    # initial evaluation no optimization
    p_loss = []
    v_loss = []
    g_loss = []
    for batch in datamodule.val_dataloader():
        with torch.no_grad():
            batch = optimizer.transfer_batch_to_device(batch, "cuda", 0)
            time_tracker.start("optimize")
            out = optimizer(batch)
            time_tracker.stop("optimize")
            optimizer.c_module.n_threshold = -1.0
            optimizer.c_module.d_threshold = 1e-02  # 1cm
            loss_info = optimizer.compute_loss(batch=batch, out=out)
            optimizer.c_module.n_threshold = n_threshold
            optimizer.c_module.d_threshold = d_threshold  # 1cm
            p_loss.append(loss_info["loss_param"])
            v_loss.append(loss_info["loss_vertices"])
            g_loss.append(loss_info["loss_geometric_point2point"])
    iters_p_loss = torch.stack(p_loss).mean().item()
    iters_g_loss = torch.stack(g_loss).mean().item()
    iters_v_loss = torch.stack(v_loss).mean().item()
    t_perf = [torch.tensor(t.time_ms) for t in list(time_tracker.tracks.values())[0]]
    iters_time = torch.stack(t_perf).min().item()
    return iters_p_loss, iters_g_loss, iters_v_loss, iters_time


def load_flame_renderer():
    # instanciate similar to training
    cfg = load_config("train", ["data=kinect"])
    K = load_intrinsics(data_dir=cfg.data.intrinsics_dir, return_tensor="pt")
    camera = Camera(
        K=K,
        width=cfg.data.width,
        height=cfg.data.height,
        near=cfg.data.near,
        far=cfg.data.far,
        scale=cfg.data.scale,
    )
    rasterizer = Rasterizer(width=camera.width, height=camera.height)
    renderer = Renderer(rasterizer=rasterizer, camera=camera)
    flame = hydra.utils.instantiate(cfg.model)
    return flame, renderer


def load_neural_optimizer(flame, renderer, path, override=[]):
    o = ["data=kinect"] + override
    cfg = load_config("train", o)
    correspondence = hydra.utils.instantiate(cfg.correspondence)
    weighting = hydra.utils.instantiate(cfg.weighting)
    residuals = hydra.utils.instantiate(cfg.residuals)
    regularize = hydra.utils.instantiate(cfg.regularize)
    neural_optimizer = NeuralOptimizer.load_from_checkpoint(
        path,
        renderer=renderer,
        flame=flame,
        correspondence=correspondence,
        regularize=regularize,
        residuals=residuals,
        weighting=weighting,
    )
    return neural_optimizer


def load_icp_optimizer(flame, renderer, overrides):
    o = ["data=kinect", "optimizer.output_dir=none"] + overrides
    cfg = load_config("train", o)
    correspondence = hydra.utils.instantiate(cfg.correspondence)
    weighting = hydra.utils.instantiate(cfg.weighting)
    residuals = hydra.utils.instantiate(cfg.residuals)
    optimizer = hydra.utils.instantiate(cfg.optimizer)
    regularize = hydra.utils.instantiate(cfg.regularize)
    icp_optimizer = hydra.utils.instantiate(
        cfg.framework,
        flame=flame,
        logger=None,
        renderer=renderer,
        correspondence=correspondence,
        regularize=regularize,
        residuals=residuals,
        optimizer=optimizer,
        weighting=weighting,
    )
    return icp_optimizer.to("cuda")


# setup the datamodule
def load_datamodule(renderer, start_frame, end_frame, jump_size):
    cfg = load_config("train", ["data=kinect"])
    datamodule = hydra.utils.instantiate(
        cfg.data,
        renderer=renderer,
        val_dataset=dict(
            start_frame=start_frame,
            end_frame=end_frame,
            jump_size=jump_size,
        ),
    )
    datamodule.setup("fit")
    return datamodule

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [8]:
N = 2
step_size = 0.7
start_frame = 48 
end_frame = 56
# start_frame = 30
# end_frame = None

# checkpoints
ours = "/home/borth/GuidedResearch/logs/2024-10-14/12-28-11_train_wo_prior__face2face_wo_landmarks/checkpoints/last.ckpt"
ours_syn = "/home/borth/GuidedResearch/logs/2024-10-14/12-28-11_train_wo_prior__face2face_wo_landmarks/checkpoints/last.ckpt"

# loadings
times = defaultdict(dict)
p_losses = defaultdict(dict)
g_losses = defaultdict(dict)
v_losses = defaultdict(dict)
flame, renderer = load_flame_renderer()

for jump_size in [1, 2, 4, 8]:
    datamodule = load_datamodule(renderer, start_frame, end_frame, jump_size)

    override = ["residuals=face2face_wo_landmarks", "regularize=dummy"]
    optimizer = load_neural_optimizer(flame, renderer, ours, override)
    optimizer.optimizer.step_size = step_size
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "ours"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss    

    override = ["residuals=face2face_wo_landmarks", "regularize=dummy"]
    optimizer = load_neural_optimizer(flame, renderer, ours_syn, override)
    optimizer.optimizer.step_size = step_size
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "ours_syn"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, [])
    optimizer.optimizer.step_size = 0.0
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=1)
    key = "base"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, ["residuals=point2plane", "weighting=dummy", "regularize=dummy"])
    optimizer.optimizer.step_size = step_size
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "icp-geo"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, ["residuals=face2face_wo_landmarks", "weighting=dummy", "regularize=dummy"])
    optimizer.optimizer.step_size = step_size
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "icp-geo+reg"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, ["residuals=face2face_wo_landmarks", "weighting=dummy", "regularize=dummy"])
    optimizer.optimizer.step_size = 0.3
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "icp-geo+reg+step"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

Creating GL context for cuda device 0
Successfully initialized EGL version 1.5
Successfully initialized OpenGL version 4.6.0 NVIDIA 535.183.01


In [9]:
desired_order = ["base", "icp-geo", "icp-geo+reg","icp-geo+reg+step", "ours"]

# Create the DataFrame for p_losses
p_losses_df = pd.DataFrame(p_losses).transpose()
p_losses_df.columns = [f"0->{c}" for c in p_losses_df.columns]
p_losses_df = p_losses_df.reindex(desired_order)
p_losses_df.columns = pd.MultiIndex.from_tuples([("FLAME (norm)", c) for c in p_losses_df.columns])

# Create the DataFrame for g_losses
g_losses_df = pd.DataFrame(g_losses).transpose()
g_losses_df.columns = [f"0->{c}" for c in g_losses_df.columns]
g_losses_df = g_losses_df.reindex(desired_order)
g_losses_df.columns = pd.MultiIndex.from_tuples([("P2P (mm)", c) for c in g_losses_df.columns])

# Create the DataFrame for v_losses
v_losses_df = pd.DataFrame(v_losses).transpose()
v_losses_df.columns = [f"0->{c}" for c in v_losses_df.columns]
v_losses_df = v_losses_df.reindex(desired_order)
v_losses_df.columns = pd.MultiIndex.from_tuples([("Vertices (mm)", c) for c in v_losses_df.columns])

# Time
time_df = pd.DataFrame(times).transpose()
time_df = pd.DataFrame(time_df.mean(axis=1))
time_df = time_df.reindex(desired_order)
time_df.columns = pd.MultiIndex.from_tuples([("Time (ms)", "")])

# Concatenate the two DataFrames
pd.concat([p_losses_df, g_losses_df, v_losses_df, time_df], axis=1)

FLAME (norm)                                P2P (mm)  \
                         0->1      0->2      0->4      0->8      0->1   
base                 0.323893  0.500221  0.599251  0.588814  1.963086   
icp-geo              1.894712  2.096590  2.485184  4.493804  1.654865   
icp-geo+reg          0.480108  0.517827  0.544179  0.581835  1.749189   
icp-geo+reg+step     0.342049  0.441612  0.506009  0.510001  1.795687   
ours                 0.295752  0.366538  0.430607  0.483300  1.669349   

                                               Vertices (mm)            \
                      0->2      0->4      0->8          0->1      0->2   
base              2.236378  2.534501  2.806092      6.464082  7.183959   
icp-geo           1.711433  1.777805  1.912375      6.540590  6.698123   
icp-geo+reg       1.763426  1.812692  1.924070      6.073561  6.093991   
icp-geo+reg+step  1.940052  2.138531  2.285906      6.114760  6.365917   
ours              1.690493  1.741028  1.844514      6.086504  6.135041   

                                       Time (ms)  
                      0->4       0->8             
base              8.352511   9.702329  25.234878  
icp-geo           7.235021  10.177958  41.016699  
icp-geo+reg       6.244667   8.344301  48.287510  
icp-geo+reg+step  6.991377   8.702782  48.334837  
ours              6.289527   8.573379  54.749609